In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../src")

In [3]:
import numpy as np
from pydrake.all import (
    StartMeshcat,
    Simulator
)

In [4]:
from world import make_n_quadrotor_system
from util import DisableCollisionChecking
from stabilization import find_fixed_point_snopt, lqr_stabilize_to_point, add_controller_to_system

In [5]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [6]:
np.random.seed(43)
n_quadrotors = 3

In [7]:
diagram, plant = make_n_quadrotor_system(meshcat, n_quadrotors)

In [8]:
fixed_point, fixed_control = find_fixed_point_snopt(diagram)
print(fixed_point)
print(fixed_control)
fixed_point.shape

True
[ 6.22426424e+00  1.45961354e+00  2.46150230e+00 -1.01269603e+01
 -4.98904960e-01 -2.12334926e+00  1.55464844e+00  3.22990204e-01
  4.33311779e+00 -5.82762175e+00 -1.28644774e+01 -2.75179152e-01
  2.03558927e+00 -2.44788441e+00  4.71484060e+00 -4.16798401e+00
  2.17283463e+01 -2.60936519e+00  2.05500850e-14 -1.26217745e-29
 -2.98516609e-14  6.31088724e-30  3.53227839e+00 -3.10719733e-01
  2.77737255e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.66726591e+01
 -4.37696905e+00 -4.27628293e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]
[-3.46910894 -3.46910894 -3.46910894 -3.46910894  3.03526216  3.03526216
  3.03526216  3.03526216  4.52289514  4.52289514  4.52289514  4.52289514]


(49,)

In [9]:
Q_quadrotor = [10.0] * 6 + [1.0] * 6
Q_freebody = [10.] * 13
Q = np.diag(Q_quadrotor * n_quadrotors + Q_freebody)
R = np.eye(4 * n_quadrotors)
lqr_controller = lqr_stabilize_to_point(diagram, fixed_point, fixed_control, Q, R)

controlled_diagram, controlled_plant = add_controller_to_system(diagram, lqr_controller)

In [10]:
simulator = Simulator(controlled_diagram)
simulator.set_target_realtime_rate(1)
context = simulator.get_mutable_context()

sg = diagram.GetSubsystemByName("scene_graph")
DisableCollisionChecking(sg, context)

simulator.set_target_realtime_rate(0.1)
# Simulate
while True:
    context.SetTime(0.0)
    context.SetContinuousState(
        fixed_point
    )
    simulator.Initialize()
    simulator.AdvanceTo(1)



RuntimeError: Spatial force applied on body base_link contains NaN.